# Exercise List 6

Utilizando o Apache Spark e demais ferramentas correlatas, além do grafo construído na lista 5, implemente as seguintes operações:

1. Determine o nó central através do grau.
2. Determine o nó central por centralidade utilizando a distância de Wasserman e a distância harmônica (consultar material).
3. Determine o nó central por intermediação.

**Setup**

In [1]:
from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

conf = SparkConf().setAppName('appName').setMaster('local')
sc = SparkContext.getOrCreate(conf)
spark = SparkSession(sc)

**Crie um CSV para armazenar as cidades, com: `id` (nome da cidade), `latitude`, `longitude` e `população`**

In [2]:
cidades = spark.read.format("csv").option("header", "true").load("data/transport/transport-nodes.csv")
cidades.show()

+---------+--------+---------+----------+
|       id|latitude|longitude|population|
+---------+--------+---------+----------+
|Araripina|  7.5766|  40.4976|     84418|
|  Caruaru|  8.2760|  35.9819|    277982|
| Igarassu|   78292|   349016|     91953|
|  Cabrobo|  8.5082|  39.3103|     33856|
|  Carpina|  7.8450|  35.2437|     81054|
| Ouricuri|  7.8809|  40.0810|     69459|
|  Surubim|  7.8543|  35.7630|     64520|
|Petrolina|  9.3891|  40.5031|       217|
|Salgueiro|  8.0725|  39.1268|     59769|
|   Recife|  8.0522|  34.9286|   1555000|
+---------+--------+---------+----------+



**Crie outro CSV para armazenar a distância entre essas cidades, com: `src`, `dst` e `relationship` como campos**

In [3]:
distancias = spark.read.format("csv").option("header", "true").load("data/transport/transport-relationships.csv")
distancias.show(30)

+---------+---------+------------+----+
|      src|      dst|relationship|cost|
+---------+---------+------------+----+
|Araripina|  Caruaru|       EROAD| 551|
|Araripina| Igarassu|       EROAD| 700|
|Araripina|  Cabrobo|       EROAD| 186|
|   Recife|  Surubim|       EROAD| 118|
|   Recife|Salgueiro|       EROAD| 510|
|   Recife|  Caruaru|       EROAD| 510|
| Ouricuri| Igarassu|       EROAD| 641|
| Ouricuri|  Surubim|       EROAD| 525|
| Ouricuri|Salgueiro|       EROAD| 359|
|  Cabrobo| Ouricuri|       EROAD| 127|
|  Cabrobo|  Carpina|       EROAD| 502|
|  Cabrobo|   Recife|       EROAD| 526|
| Igarassu|Salgueiro|       EROAD| 529|
| Igarassu| Ouricuri|       EROAD| 197|
| Igarassu|   Recife|       EROAD|  28|
|  Carpina|Petrolina|       EROAD| 681|
|  Carpina|Salgueiro|       EROAD| 480|
|  Carpina| Ouricuri|       EROAD| 591|
|Petrolina| Ouricuri|       EROAD| 154|
|Petrolina|  Surubim|       EROAD|  66|
|Petrolina| Igarassu|       EROAD| 726|
|  Surubim|Araripina|       EROAD| 583|


**Utilizando as bibliotecas do Spark, crie um objeto GraphFrame a partir desses dois CSVs.**

In [6]:
from graphframes import *

g = GraphFrame(cidades, distancias)

In [7]:
g.vertices.show()

+---------+--------+---------+----------+
|       id|latitude|longitude|population|
+---------+--------+---------+----------+
|Araripina|  7.5766|  40.4976|     84418|
|  Caruaru|  8.2760|  35.9819|    277982|
| Igarassu|   78292|   349016|     91953|
|  Cabrobo|  8.5082|  39.3103|     33856|
|  Carpina|  7.8450|  35.2437|     81054|
| Ouricuri|  7.8809|  40.0810|     69459|
|  Surubim|  7.8543|  35.7630|     64520|
|Petrolina|  9.3891|  40.5031|       217|
|Salgueiro|  8.0725|  39.1268|     59769|
|   Recife|  8.0522|  34.9286|   1555000|
+---------+--------+---------+----------+



In [8]:
g.edges.show()

+---------+---------+------------+----+
|      src|      dst|relationship|cost|
+---------+---------+------------+----+
|Araripina|  Caruaru|       EROAD| 551|
|Araripina| Igarassu|       EROAD| 700|
|Araripina|  Cabrobo|       EROAD| 186|
|   Recife|  Surubim|       EROAD| 118|
|   Recife|Salgueiro|       EROAD| 510|
|   Recife|  Caruaru|       EROAD| 510|
| Ouricuri| Igarassu|       EROAD| 641|
| Ouricuri|  Surubim|       EROAD| 525|
| Ouricuri|Salgueiro|       EROAD| 359|
|  Cabrobo| Ouricuri|       EROAD| 127|
|  Cabrobo|  Carpina|       EROAD| 502|
|  Cabrobo|   Recife|       EROAD| 526|
| Igarassu|Salgueiro|       EROAD| 529|
| Igarassu| Ouricuri|       EROAD| 197|
| Igarassu|   Recife|       EROAD|  28|
|  Carpina|Petrolina|       EROAD| 681|
|  Carpina|Salgueiro|       EROAD| 480|
|  Carpina| Ouricuri|       EROAD| 591|
|Petrolina| Ouricuri|       EROAD| 154|
|Petrolina|  Surubim|       EROAD|  66|
+---------+---------+------------+----+
only showing top 20 rows



### 1. Determine o nó central através do grau.

In [9]:
total_degree = g.degrees
in_degree    = g.inDegrees
out_degree   = g.outDegrees

In [10]:
(total_degree
    .join(in_degree, "id", how="left")
    .join(out_degree, "id", how="left")
    .fillna(0)
    .sort("inDegree", ascending=False)
    .show())

+---------+------+--------+---------+
|       id|degree|inDegree|outDegree|
+---------+------+--------+---------+
| Ouricuri|     8|       5|        3|
|Salgueiro|     8|       5|        3|
| Igarassu|     7|       4|        3|
|Petrolina|     6|       3|        3|
|  Caruaru|     6|       3|        3|
|  Surubim|     6|       3|        3|
|   Recife|     5|       2|        3|
|Araripina|     5|       2|        3|
|  Cabrobo|     5|       2|        3|
|  Carpina|     4|       1|        3|
+---------+------+--------+---------+



Por grau, os nós centrais são: Ouricuri e Salgueiro.

### 2. Determine o nó central por centralidade utilizando a distância de Wasserman e a distância harmônica (consultar material).

In [33]:
import sys

# install a pip package in the current Jupyter kernel
!{sys.executable} -m pip install networkx

In [34]:
import networkx as nx
import pandas as pd
import json

In [35]:
df = pd.read_csv('data/transport/transport-relationships-networkx.csv')

In [36]:
Graphtype = nx.DiGraph()
gnx = nx.from_pandas_edgelist(df, edge_attr='cost', create_using=Graphtype)

In [44]:
def run_algorithm(runnable, parameter, return_max_only=True):
    result = runnable(parameter)
    print(json.dumps(result, indent=2))
    return result if (return_max_only == False) else max(result, key=result.get)

**Distância de [Wasserman](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.closeness_centrality.html?highlight=wasserman)**

In [51]:
run_algorithm(nx.closeness_centrality, gnx)

{
  "Araripina": 0.5,
  "Caruaru": 0.5625,
  "Igarassu": 0.6428571428571429,
  "Cabrobo": 0.5,
  "Recife": 0.5,
  "Surubim": 0.5625,
  "Salgueiro": 0.6923076923076923,
  "Ouricuri": 0.6923076923076923,
  "Carpina": 0.36,
  "Petrolina": 0.6
}


'Salgueiro'

**Distancia [Harmônica](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.harmonic_centrality.html?highlight=harmonic#networkx.algorithms.centrality.harmonic_centrality)**

In [47]:
run_algorithm(nx.harmonic_centrality, gnx)

{
  "Araripina": 5.166666666666666,
  "Caruaru": 5.833333333333333,
  "Igarassu": 6.5,
  "Cabrobo": 5.166666666666666,
  "Recife": 5.166666666666666,
  "Surubim": 5.833333333333333,
  "Salgueiro": 7.0,
  "Ouricuri": 7.0,
  "Carpina": 3.833333333333334,
  "Petrolina": 6.0
}


'Salgueiro'

### 3. Determine o nó central por intermediação ([betweenness](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.bipartite.centrality.betweenness_centrality.html?highlight=betweenness#networkx.algorithms.bipartite.centrality.betweenness_centrality))

In [52]:
run_algorithm(nx.betweenness_centrality, gnx)

{
  "Araripina": 0.06018518518518519,
  "Caruaru": 0.08912037037037036,
  "Igarassu": 0.12384259259259257,
  "Cabrobo": 0.1597222222222222,
  "Recife": 0.05648148148148147,
  "Surubim": 0.09583333333333334,
  "Salgueiro": 0.2710648148148148,
  "Ouricuri": 0.11435185185185186,
  "Carpina": 0.021296296296296292,
  "Petrolina": 0.0636574074074074
}


'Salgueiro'

**Extra: Determine o nó central por [PageRank](https://graphframes.github.io/graphframes/docs/_site/user-guide.html#pagerank)**

In [31]:
results = g.pageRank(resetProbability=0.15,
                     maxIter=20)

In [32]:
results.vertices.sort("pagerank", ascending=False).show()

+---------+--------+---------+----------+-------------------+
|       id|latitude|longitude|population|           pagerank|
+---------+--------+---------+----------+-------------------+
|Salgueiro|  8.0725|  39.1268|     59769| 1.5366951248264467|
| Ouricuri|  7.8809|  40.0810|     69459|  1.396244031087945|
| Igarassu|   78292|   349016|     91953|  1.320543664197592|
|  Surubim|  7.8543|  35.7630|     64520| 1.0306770214526706|
|Petrolina|  9.3891|  40.5031|       217| 0.9515740788711936|
|  Caruaru|  8.2760|  35.9819|    277982| 0.9060900588412013|
|Araripina|  7.5766|  40.4976|     84418| 0.8774219616291241|
|  Cabrobo|  8.5082|  39.3103|     33856| 0.8339998727525862|
|   Recife|  8.0522|  34.9286|   1555000| 0.7604541089661059|
|  Carpina|  7.8450|  35.2437|     81054|0.38630007737513467|
+---------+--------+---------+----------+-------------------+



Nó central utilizando PageRank: Salgueiro.